In [16]:
import tempfile
import os.path
import subprocess
import shutil
import scipy.ndimage.filters
import numpy.random
import nibabel as nib
import nibabel.processing
import numpy as np
from glob import glob
    

def AntsAffineRegistration(scrFilename, targetFilename, transfo=None, resFilename=None , mask=None, rigid=False,cost='MI') :
    tempDir=tempfile.mkdtemp()
    srcDirname=os.path.dirname(scrFilename)+"/"
    srcBasename=os.path.basename(scrFilename)
    targetBasename=os.path.basename(targetFilename)
    srcBasename=srcBasename.split('.')[0]
    targetBasename=targetBasename.split('.')[0]
    
    if transfo == None :
        transfo=srcDirname+srcBasename+"_To_"+targetBasename+".mat"
    
    if resFilename == None :
        resFilename=srcDirname+srcBasename+"_To_"+targetBasename+".nii.gz"
    
    
    if rigid :
        regtype="Rigid[0.1]"
    else :
        regtype="Affine[0.1]"
     
    costCommandLine="CC["+targetFilename+","+scrFilename+",1,3,Regular,1]"

    if cost == "GC"  : 
        costCommandLine="GC["+targetFilename+","+scrFilename+",1]"
        
    if cost == "Mattes"  : 
        costCommandLine="Mattes["+targetFilename+","+scrFilename+"1,32,Regular,1]"
   
    if cost == "MeanSquares"  : 
        costCommandLine="MeanSquares["+targetFilename+","+scrFilename+"1,Regular,1]"
    
    command = ['antsRegistration',
                "-d", "3",
                "-m",costCommandLine,
                "-t",regtype,
                "-c","1000x1000x1000x100",
                "-f","16x8x4x1",
                "-s","8x4x2x0vox",
                "-r","["+targetFilename+","+scrFilename+",0]",
                "-v",
                "-o", os.path.join(tempDir,"res")]

    if mask is not None : 
         command.extend(["-x",mask])
    print((" ".join(command)))
    subprocess.call(command)
    shutil.copy(os.path.join(tempDir,"res0GenericAffine.mat"),transfo)
    shutil.rmtree(tempDir)

    ApplyAntsRegistration(scrFilename, targetFilename, transfo, resFilename) 
    
def ApplyAntsRegistration(scrFilename, targetFilename, listTransfo, resFilename, interp="Linear") :
    
    if type(listTransfo) != list :
        listTransfo=[listTransfo]
    
    command = ['antsApplyTransforms',
               "-d","3",
               "-v","1",
               "-e","0",
               "-i",scrFilename,
               "-o",resFilename,
               "-r", targetFilename,"-n",interp,"-t"]
    for trf in reversed(listTransfo) : 
        command.append(trf)
        
    print((" ".join(command)))
    subprocess.call(command)

In [17]:
ref = os.path.join("/home/valentini/dev/Mousenet/dataset2/Atlas/", "Identity_Neatex_MRI_A9.nii.gz")
for dataset in ['Neatex',
                #"Feminad",
                #"Painfact",
                #"IRIS",
               ]:
    data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset2/" + str(dataset) + "/")    
    mris = sorted(glob(os.path.join(data_dir, 'MRI_N4_Resample_Norm_Identity', "*.nii.gz")))        
    for i, mri in enumerate(mris):
        print(mris[i])
        print(ref)
        print('-'*5)
        AntsAffineRegistration(mris[i], ref, cost='MI')
    print('-'*40)
    break

/home/valentini/dev/Mousenet/dataset2/Neatex/MRI_N4_Resample_Norm_Identity/Identity_Norm_N4_Truncated_MRI_A0.nii.gz
/home/valentini/dev/Mousenet/dataset2/Atlas/Identity_Neatex_MRI_A9.nii.gz
-----
antsRegistration -d 3 -m CC[/home/valentini/dev/Mousenet/dataset2/Atlas/Identity_Neatex_MRI_A9.nii.gz,/home/valentini/dev/Mousenet/dataset2/Neatex/MRI_N4_Resample_Norm_Identity/Identity_Norm_N4_Truncated_MRI_A0.nii.gz,1,3,Regular,1] -t Affine[0.1] -c 1000x1000x1000x100 -f 16x8x4x1 -s 8x4x2x0vox -r [/home/valentini/dev/Mousenet/dataset2/Atlas/Identity_Neatex_MRI_A9.nii.gz,/home/valentini/dev/Mousenet/dataset2/Neatex/MRI_N4_Resample_Norm_Identity/Identity_Norm_N4_Truncated_MRI_A0.nii.gz,0] -v -o /tmp/tmp9spyyqd5/res
All_Command_lines_OK
Using double precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Image center alignment using fixed image: /home/valentini/dev/Mousenet/dataset2/Atlas/Identity_Neatex_MRI_A9.nii.gz and moving image: /home/valen

DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 2DIAGNOSTIC,     1, -9.444701964035e-01, 1.797693134862e+308, 8.6128e-01, 1.6157e-01, 
 2DIAGNOSTIC,     2, -9.466369391326e-01, 1.797693134862e+308, 9.8086e-01, 1.1957e-01, 
 2DIAGNOSTIC,     3, -9.500373474348e-01, 1.797693134862e+308, 1.1020e+00, 1.2112e-01, 
 2DIAGNOSTIC,     4, -9.550886870516e-01, 1.797693134862e+308, 1.2326e+00, 1.3058e-01, 
 2DIAGNOSTIC,     5, -9.618575227039e-01, 1.797693134862e+308, 1.3512e+00, 1.1861e-01, 
 2DIAGNOSTIC,     6, -9.694198952630e-01, 1.797693134862e+308, 1.4689e+00, 1.1775e-01, 
 2DIAGNOSTIC,     7, -9.772190067654e-01, 1.797693134862e+308, 1.6559e+00, 1.8696e-01, 
 2DIAGNOSTIC,     8, -9.778960640620e-01, 1.797693134862e+308, 1.7973e+00, 1.4142e-01, 
 2DIAGNOSTIC,     9, -9.779227385157e-01, 1.797693134862e+308, 2.0140e+00, 2.1670e-01, 
 2DIAGNOSTIC,    10, -9.779230719881e-01, 2.713789557197e-03, 2.2255e+00, 2.1154e-01, 
 2DIAGNOSTIC,    11, -9.779230787522e-0

 2DIAGNOSTIC,    31, -6.931380417232e-01, 1.027118221829e-06, 2.7051e+03, 7.4068e+01, 
  Elapsed time (stage 0): 2.7176e+03


Total elapsed time: 2.7176e+03
antsApplyTransforms -d 3 -v 1 -e 0 -i /home/valentini/dev/Mousenet/dataset2/Neatex/MRI_N4_Resample_Norm_Identity/Identity_Norm_N4_Truncated_MRI_A0.nii.gz -o /home/valentini/dev/Mousenet/dataset2/Neatex/MRI_N4_Resample_Norm_Identity/Identity_Norm_N4_Truncated_MRI_A0_To_Identity_Neatex_MRI_A9.nii.gz -r /home/valentini/dev/Mousenet/dataset2/Atlas/Identity_Neatex_MRI_A9.nii.gz -n Linear -t /home/valentini/dev/Mousenet/dataset2/Neatex/MRI_N4_Resample_Norm_Identity/Identity_Norm_N4_Truncated_MRI_A0_To_Identity_Neatex_MRI_A9.mat
Using double precision for computations.
Input scalar image: /home/valentini/dev/Mousenet/dataset2/Neatex/MRI_N4_Resample_Norm_Identity/Identity_Norm_N4_Truncated_MRI_A0.nii.gz
Reference image: /home/valentini/dev/Mousenet/dataset2/Atlas/Identity_Neatex_MRI_A9.nii.gz
The composite transform comprises the following

 2DIAGNOSTIC,    28, -9.430885745146e-01, 1.151536272434e-06, 5.9339e-01, 1.8783e-02, 
 2DIAGNOSTIC,    29, -9.430885745146e-01, 1.111306385179e-06, 6.0800e-01, 1.4614e-02, 
 2DIAGNOSTIC,    30, -9.430885745146e-01, 1.073792540544e-06, 6.2265e-01, 1.4647e-02, 
 2DIAGNOSTIC,    31, -9.430885745146e-01, 1.038728667635e-06, 6.3727e-01, 1.4624e-02, 
 2DIAGNOSTIC,    32, -9.430885745146e-01, 1.005882349962e-06, 6.5198e-01, 1.4708e-02, 
DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 2DIAGNOSTIC,     1, -8.432260303771e-01, 1.797693134862e+308, 8.0546e-01, 1.5349e-01, 
 2DIAGNOSTIC,     2, -8.439594729856e-01, 1.797693134862e+308, 9.1922e-01, 1.1376e-01, 
 2DIAGNOSTIC,     3, -8.450897215331e-01, 1.797693134862e+308, 1.0360e+00, 1.1679e-01, 
 2DIAGNOSTIC,     4, -8.468736452518e-01, 1.797693134862e+308, 1.1527e+00, 1.1665e-01, 
 2DIAGNOSTIC,     5, -8.501486047614e-01, 1.797693134862e+308, 1.2674e+00, 1.1478e-01, 
 2DIAGNOSTIC,     6, -8.541728761922e-01, 1

KeyboardInterrupt: 